In [6]:
# 필요한 라이브러리를 iport 

import math
import numpy as np
import cv2
import matplotlib.pyplot as plt
import imutils

## FFT: the better accuracy

+ image: Our input image for blur detection
+ size: The size of the radius around the centerpoint of the image for which we will zero out the FFT shift
+ thresh: A value which the mean value of the magnitudes (more on that later) will be compared to for determining whether an image is considered blurry or not blurry
+ vis: A boolean indicating whether to visualize/plot the original input image and magnitude image using matplotlib

In [49]:
def detect_blur(img, size=60, thresh=10, vis=False):
    (h, w) = img.shape #height, width
    (cX, cY) = (int(w / 2.0), int(h / 2.0))
    
    fft = np.fft.fft2(img) #coupute fft
    fftShift = np.fft.fftshift(fft)
    
    if vis:
        magnitude = 20*np.log(np.abs(fftShift))
        
        (fig, ax) = plt.subplots(1,2, )
        ax[0].imshow(image, cmap = 'gray')
        ax[0].set_title("input")
        ax[0].set_xticks([])
        ax[0].set_yticks([])
        
        (fig, ax) = plt.subplots(1,2, )
        ax[1].imshow(image, cmap = 'gray')
        ax[1].set_title("Magnitude Spectrum")
        ax[1].set_xticks([])
        ax[1].set_yticks([])
        
        plt.show()
        
    fftShift[cY - size:cY + size, cX - size:cX + size] = 0
    fftShift = np.fft.ifftshift(fftShift)
    recon = np.fft.ifft2(fftShift)
    
    magnitude = 20 * np.log(np.abs(recon))
    mean = np.mean(magnitude)
    
    return (mean, mean <= thresh)

In [56]:
#image
img = cv2.imread('out_of_focus0080.jpg')
img = imutils.resize(img, width=500)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

(mean, blurry) = detect_blur(gray, size=60, thresh=10, vis=False)
image = np.dstack([gray]*3)

color = (0, 0, 255) if blurry else (0,255,0)
text = "Blurry ({:.4f})" if blurry else "Not Blurry ({:.4f})"

text = text.format(mean)

cv2.putText(image, text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
print("[INFO] {}".format(text))

cv2.imshow("Output", image)
cv2.waitKey(0)
# cv2.DestroyAllWindows()

[INFO] Blurry (-3.6660)


-1

In [15]:
print(mean, blurry)

18.365471906119183 False


In [ ]:
# check to see if are going to test our FFT blurriness detector using
# various sizes of a Gaussian kernel
if args["test"] > 0:
    
    # loop over various blur radii
    for radius in range(1, 30, 2):
        
        # clone the original grayscale image
        image = gray.copy()
        
        # check to see if the kernel radius is greater than zero
        if radius > 0:
            
            # blur the input image by the supplied radius using a
            # Gaussian kernel
            image = cv2.GaussianBlur(image, (radius, radius), 0)
            # apply our blur detector using the FFT
            (mean, blurry) = detect_blur_fft(image, size=60,
                thresh=args["thresh"], vis=args["vis"] > 0)
            # draw on the image, indicating whether or not it is blurry

            image = np.dstack([image] * 3)
            color = (0, 0, 255) if blurry else (0, 255, 0)
            text = "Blurry ({:.4f})" if blurry else "Not Blurry ({:.4f})"
            text = text.format(mean)
            cv2.putText(image, text, (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
            print("[INFO] Kernel: {}, Result: {}".format(radius, text))
            
        # show the image
        cv2.imshow("Test Image", image)
        cv2.waitKey(0)

## 참고 -1
반사광이 심할 때 주파수 필터를 이용하여 줄임.

In [2]:
# 반사광이 심할 때 줄여줌. 주파수 필터를 이용.

### homomorphic filter는 gray scale image에 대해서 밖에 안 되므로
### YUV color space로 converting한 뒤 Y에 대해 연산을 진행
img = cv2.imread('out_of_focus0035.jpg')
img_YUV = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)  

y = img_YUV[:,:,0]
rows = y.shape[0]    
cols = y.shape[1]

### illumination elements와 reflectance elements를 분리하기 위해 log를 취함
imgLog = np.log1p(np.array(y, dtype='float') / 255) # y값을 0~1사이로 조정한 뒤 log(x+1)
 
### frequency를 이미지로 나타내면 4분면에 대칭적으로 나타나므로 
### 4분면 중 하나에 이미지를 대응시키기 위해 row와 column을 2배씩 늘려줌
M = 2*rows + 1
N = 2*cols + 1
 
### gaussian mask 생성 sigma = 10
sigma = 10
(X, Y) = np.meshgrid(np.linspace(0, N-1, N), np.linspace(0, M-1, M)) # 0~N-1(and M-1) 까지 1단위로 space를 만듬
Xc = np.ceil(N/2) # 올림 연산
Yc = np.ceil(M/2)
gaussianNumerator = (X - Xc)**2 + (Y - Yc)**2 # 가우시안 분자 생성


### low pass filter와 high pass filter 생성
LPF = np.exp(-gaussianNumerator / (2*sigma*sigma))
HPF = 1 - LPF
 
### LPF랑 HPF를 0이 가운데로 오도록iFFT함. 
### 사실 이 부분이 잘 이해가 안 가는데 plt로 이미지를 띄워보니 shuffling을 수행한 효과가 났음
### 에너지를 각 귀퉁이로 모아 줌
LPF_shift = np.fft.ifftshift(LPF.copy())
HPF_shift = np.fft.ifftshift(HPF.copy())

### Log를 씌운 이미지를 FFT해서 LPF와 HPF를 곱해 LF성분과 HF성분을 나눔
img_FFT = np.fft.fft2(imgLog.copy(), (M, N))
img_LF = np.real(np.fft.ifft2(img_FFT.copy() * LPF_shift, (M, N)))
img_HF = np.real(np.fft.ifft2(img_FFT.copy() * HPF_shift, (M, N)))


### 각 LF, HF 성분에 scaling factor를 곱해주어 조명값과 반사값을 조절함
gamma1 = 0.3
gamma2 = 1.5
img_adjusting = gamma1*img_LF[0:rows, 0:cols] + gamma2*img_HF[0:rows, 0:cols]

### 조정된 데이터를 이제 exp 연산을 통해 이미지로 만들어줌
img_exp = np.expm1(img_adjusting) # exp(x) + 1
img_exp = (img_exp - np.min(img_exp)) / (np.max(img_exp) - np.min(img_exp)) # 0~1사이로 정규화
img_out = np.array(255*img_exp, dtype = 'uint8') # 255를 곱해서 intensity값을 만들어줌

In [8]:
### 마지막으로 YUV에서 Y space를 filtering된 이미지로 교체해주고 RGB space로 converting
img_YUV[:,:,0] = img_out
result = cv2.cvtColor(img_YUV, cv2.COLOR_YUV2BGR)
cv2.imshow('homomorphic', result)
cv2.waitKey(0)
cv2.destroyAllWindows()